In [1]:
import sklearn
import pandas as pd

In [27]:
data_folder = '~/instacart_data/'
aisles = pd.read_csv(data_folder + 'aisles.csv')
departments = pd.read_csv(data_folder + 'departments.csv')
orders = pd.read_csv(data_folder + 'orders.csv')
orders_prior = pd.read_csv(data_folder + 'order_products__prior.csv')
orders_train = pd.read_csv(data_folder + 'order_products__train.csv')
products = pd.read_csv(data_folder + 'products.csv')
sample_submission = pd.read_csv(data_folder + 'sample_submission.csv')

orders_full was made in `make_orders_full.ipynb`

Notice that we have product id, product name, aisle id, and deparment id. The product name could be useful in the future if we wanted to do some sort of word embeddings with these names, for example we could group all the 'cookies' together or all the 'teas'. The aisle id would also be used as a way to group certain food types together since the grocery stores group similar items together. The department id, as we will see in the next cell, is another grouping of foods, such as 'frozen', 'bakery', 'produce', etc. This seems like a larger grouping system then either looking at the individual names or the aisles, since a 'frozen' department might have many aisles, one for ice cream, one for frozen diners, etc. So the hierarchy of food groupings that we might use are:

- Individual items (using the product_name)
- Aisle
- Department

In [3]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
products.shape[0]

49688

There are about 50,000 products, each with aisle_id and department_id

In [5]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [6]:
departments.shape[0]

21

There are 21 departments

In [7]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [8]:
aisles.shape[0]

134

All together we have the following statistics:
    
- products: ~50,000
- departments: 21
- aisles: 134

There are about 134 aisles

## The three 'orders' tables

There are three sets of data which have information about the orders that customers have made.

**Orders**: This table includes information about all the orders placed. The features are:
- *order_id*
- *user_id*
- *eval_set*: this feature tells which of the three sets the order belongs in (prior, train, test)
- *order_number*
- *order_dow*: not sure what this feature is
- *order_hour_of_the_day*: hour of the day the order was places (out of a 24 hour clock)
- *days_since_prior_order*

The `orders` table does not include information about the individual products that go into each order. `orders` records the relationship between a user and the order. 

The tables `orders_prior` and `orders_train` relate the individual products (represented by product ids) to the orders. These two tables also record the order in which the item was added to the order and it this item is a reorder. These tables have the following features:

- *order_id*
- *product_id*
- *add_to_cart_order*: what order the item was added to the cart. Was it the first added? The last?
- *reordered*: Has this person ordered this item in the past?

In order to get a large dataframe which includes all the information about the individual orders and the order itself we would have to concatenate the `orders_prior` and the `orders_train` table and then merge the new table with the orders table on the `order_id` column. We want to include all the orders, even if there are not products in the order so we would left join the `orders_prior + orders_train` on the `orders` table which would look something like this:

```python
df = pd.concat((orders_train, orders_prior), axis=0)
df = orders.merge(df, on='order_id', how='left')
```

Notice that every order can have multiple items associated to it. For example, take order_id 2539329. This is an order labeled as "prior" and associated to user_id 1.

In [9]:
orders_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [10]:
orders.loc[orders['user_id'] == 1]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


Notice that user 1 has multiple orders. All orders except order 1187899 are marked as prior and order 1187899 is marked as train. Each of these orders can have multiple items associated to it:

In [11]:
orders_prior.loc[orders_prior['order_id'] == 2539329]

,order_id,product_id,add_to_cart_order,reordered
24076664,2539329,196,1,0
24076665,2539329,14084,2,0
24076666,2539329,12427,3,0
24076667,2539329,26088,4,0
24076668,2539329,26405,5,0


In [12]:
orders_train.loc[orders_train['order_id'] == 2539329]

,order_id,product_id,add_to_cart_order,reordered


In [13]:
orders_train.loc[orders_train['order_id'] == 1187899]

,order_id,product_id,add_to_cart_order,reordered
484420,1187899,196,1,1
484421,1187899,25133,2,1
484422,1187899,38928,3,1
484423,1187899,26405,4,1
484424,1187899,39657,5,1
484425,1187899,10258,6,1
484426,1187899,13032,7,1
484427,1187899,26088,8,1
484428,1187899,27845,9,0
484429,1187899,49235,10,1


The prior orders are there to help predict the items in the test set.

Notice that there is no table for orders associated to the train orders because the goal of the exercise is to predict these items that should be in the test orders.

In [14]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [15]:
orders['eval_set'].unique()

array(['prior', 'train', 'test'], dtype=object)

In [16]:
orders_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [17]:
orders_prior.shape[0]

32434489

In [18]:
len(orders_prior['order_id'].unique())

3214874

There are about 32.5 million rows in the prior orders table which represent items being places in orders. THe number of orders is about 3.2 million.

In [19]:
orders_train.shape

(1384617, 4)

In [20]:
orders_train['order_id'].unique()

array([      1,      36,      38, ..., 3421058, 3421063, 3421070])

In [21]:
sample_submission.head()

,order_id,products
0,17,39276 29259
1,34,39276 29259
2,137,39276 29259
3,182,39276 29259
4,257,39276 29259


In [22]:
orders.loc[orders['eval_set'] == 'test'].head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


There are about 1.4 million rows in the training data

In [23]:
orders_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [24]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [11]:
orders_full = pd.read_csv('~/instacart_data/orders_full.csv')

MemoryError: 

In [ ]:
orders_full.head()

In [ ]:
orders_full = orders_full.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
orders_full.head()

In [6]:
orders_full.loc[orders_full['user_id'] == 2].head()

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
70,70,2168274,2,prior,1,2,11,NaN,32792.0,1.0,0.0
71,71,2168274,2,prior,1,2,11,NaN,47766.0,2.0,0.0
72,72,2168274,2,prior,1,2,11,NaN,20574.0,3.0,0.0
73,73,2168274,2,prior,1,2,11,NaN,12000.0,4.0,0.0
74,74,2168274,2,prior,1,2,11,NaN,48110.0,5.0,0.0


In [8]:
orders_full.loc[orders_full['eval_set'] == 'test']

,Unnamed: 0,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
384,384,2774568,3,test,13,5,15,11.0,NaN,NaN,NaN
403,403,329954,4,test,6,3,12,30.0,NaN,NaN,NaN
464,464,1528013,6,test,4,3,16,22.0,NaN,NaN,NaN
1086,1086,1376945,11,test,8,6,11,8.0,NaN,NaN,NaN
1161,1161,1356845,12,test,6,1,20,30.0,NaN,NaN,NaN
1541,1541,2161313,15,test,23,1,9,7.0,NaN,NaN,NaN
1612,1612,1416320,16,test,7,0,13,7.0,NaN,NaN,NaN
2167,2167,1735923,19,test,10,6,17,8.0,NaN,NaN,NaN
2190,2190,1980631,20,test,5,1,11,30.0,NaN,NaN,NaN
2471,2471,139655,22,test,16,5,6,1.0,NaN,NaN,NaN
